<a href="https://colab.research.google.com/github/Vakhranev/BraPoRus/blob/main/%D0%9C%D0%B5%D1%82%D1%80%D0%B8%D0%BA%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import os
import glob
import numpy as np
import re

# Папка с .cha файлами
folder_path = "files"
file_list = glob.glob(os.path.join(folder_path, "*.cha"))
print(f"Найдено {len(file_list)} файлов в папке {folder_path}.")

# Списки для хранения результатов
file_avg_lengths = []      # Средняя длина токена по каждому файлу
file_std_lengths = []      # Стандартное отклонение по каждому файлу
all_token_lengths = []     # Все токены всех файлов

for file_path in file_list:
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    token_lengths = []

    filename = os.path.basename(file_path)
    speaker_prefix = filename[:3]

    for line in lines:
        if line.startswith(f"*{speaker_prefix}:"):
            cleaned_line = re.sub(r"\*{0}:\s*".format(speaker_prefix), "", line)
            cleaned_line = re.sub(r"•\d+_\d+•", "", cleaned_line)
            cleaned_line = cleaned_line.strip()
            tokens = cleaned_line.split()
            token_lengths.extend([len(token) for token in tokens])

    if token_lengths:
        avg_length = np.mean(token_lengths)
        std_length = np.std(token_lengths)
        file_avg_lengths.append(avg_length)
        file_std_lengths.append(std_length)
        all_token_lengths.extend(token_lengths)
        print(f"{filename}: средняя длина токена = {avg_length:.2f}, SD = {std_length:.2f}")
    else:
        print(f"{filename}: нет данных для анализа")

# Общие метрики
if all_token_lengths:
    overall_avg = np.mean(all_token_lengths)
    overall_sd = np.std(all_token_lengths)
    print(f"\nСредняя длина токена по всем файлам: {overall_avg:.2f}")
    print(f"Стандартное отклонение длины токенов по всем файлам: {overall_sd:.2f}")
else:
    print("Нет токенов для анализа.")

Найдено 16 файлов в папке files.
TNK_S5.cha: средняя длина токена = 4.82, SD = 4.16
FFK_S1.cha: средняя длина токена = 4.59, SD = 3.82
SAP_S1.cha: средняя длина токена = 4.64, SD = 4.12
ZVH_S1.cha: средняя длина токена = 4.70, SD = 3.89
AVM_S1.cha: средняя длина токена = 4.47, SD = 3.68
SAK_S2.cha: средняя длина токена = 4.76, SD = 3.92
IMK_S3.cha: средняя длина токена = 4.77, SD = 4.21
TYL_S1.cha: средняя длина токена = 5.11, SD = 4.36
VVG_S1.cha: средняя длина токена = 5.21, SD = 4.40
ENL_S1.cha: средняя длина токена = 4.18, SD = 3.66
NVM_S3.cha: средняя длина токена = 4.68, SD = 4.03
EKS_S4.cha: средняя длина токена = 4.93, SD = 4.14
GAA_S1.cha: средняя длина токена = 4.32, SD = 3.74
AVG_S2.cha: средняя длина токена = 5.06, SD = 4.26
LNI_S5.cha: средняя длина токена = 4.28, SD = 3.47
EAB_S9.cha: средняя длина токена = 4.42, SD = 3.65

Средняя длина токена по всем файлам: 4.65
Стандартное отклонение длины токенов по всем файлам: 3.96


In [4]:
!pip install pymorphy3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 66.3 MB/s eta 0:00:00


In [12]:
import os
import glob
import numpy as np
import re
import pymorphy3

morph = pymorphy3.MorphAnalyzer()
folder_path = "files"  # Укажи правильный путь к папке с .cha файлами
file_list = glob.glob(os.path.join(folder_path, "*.cha"))

vowels = set('аеёиоуыэюя')
all_vowel_counts = []
file_results = []

for file_path in file_list:
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    vowel_counts = []

    filename = os.path.basename(file_path)
    speaker_prefix = filename[:3]

    for line in lines:
        if line.startswith(f"*{speaker_prefix}:"):
            cleaned_line = re.sub(rf"\*{speaker_prefix}:\s*", "", line)
            cleaned_line = re.sub(r"•\d+_\d+•", "", cleaned_line)
            cleaned_line = cleaned_line.strip()

            tokens = cleaned_line.split()

            for token in tokens:
                parsed = morph.parse(token)
                if parsed:
                    normal_form = parsed[0].normal_form
                    vowels_in_token = sum(1 for ch in normal_form.lower() if ch in vowels)
                    vowel_counts.append(vowels_in_token)

    if vowel_counts:
        avg_vowels = np.mean(vowel_counts)
        std_vowels = np.std(vowel_counts)
        file_results.append((filename, avg_vowels, std_vowels))
        all_vowel_counts.extend(vowel_counts)
    else:
        file_results.append((filename, 0, 0))

# Вывод по каждому файлу
for fname, avg_vow, std_vow in file_results:
    print(f"{fname}: среднее число гласных = {avg_vow:.2f}, SD = {std_vow:.2f}")

# Общие показатели по всем файлам
if all_vowel_counts:
    print(f"\nОбщее среднее число гласных в токене: {np.mean(all_vowel_counts):.2f}")
    print(f"Общее стандартное отклонение числа гласных: {np.std(all_vowel_counts):.2f}")
else:
    print("Нет данных для анализа.")

TNK_S5.cha: среднее число гласных = 1.28, SD = 1.22
FFK_S1.cha: среднее число гласных = 1.30, SD = 1.13
SAP_S1.cha: среднее число гласных = 1.26, SD = 1.15
ZVH_S1.cha: среднее число гласных = 1.31, SD = 1.16
AVM_S1.cha: среднее число гласных = 1.35, SD = 1.15
SAK_S2.cha: среднее число гласных = 1.34, SD = 1.10
IMK_S3.cha: среднее число гласных = 1.20, SD = 1.12
TYL_S1.cha: среднее число гласных = 1.24, SD = 1.19
VVG_S1.cha: среднее число гласных = 1.25, SD = 1.25
ENL_S1.cha: среднее число гласных = 1.21, SD = 1.07
NVM_S3.cha: среднее число гласных = 1.25, SD = 1.19
EKS_S4.cha: среднее число гласных = 1.29, SD = 1.13
GAA_S1.cha: среднее число гласных = 1.24, SD = 1.11
AVG_S2.cha: среднее число гласных = 1.36, SD = 1.29
LNI_S5.cha: среднее число гласных = 1.26, SD = 1.09
EAB_S9.cha: среднее число гласных = 1.34, SD = 1.08

Общее среднее число гласных в токене: 1.28
Общее стандартное отклонение числа гласных: 1.15


In [13]:
import os
import glob
import re
import numpy as np

# Путь к папке с .cha файлами
folder_path = "files"
file_list = glob.glob(os.path.join(folder_path, "*.cha"))
print(f"Найдено {len(file_list)} файлов в папке {folder_path}.")

# Загрузка словаря морфем
morpheme_dictionary = {}
with open("cleaned_tihonov (5) (1).txt", 'r', encoding='utf-8') as dict_file:
    for line in dict_file:
        line = line.strip()
        if ' ' in line:
            word, morphemes = line.split(' ', 1)
            morpheme_count = morphemes.count('/') + 1
            morpheme_dictionary[word.lower()] = morpheme_count

# Основной цикл по файлам
file_results = []
all_morpheme_counts = []

for file_path in file_list:
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    filename = os.path.basename(file_path)
    speaker_prefix = filename[:3]

    morpheme_counts_in_file = []

    for line in lines:
        if line.startswith(f"*{speaker_prefix}:"):
            cleaned_line = re.sub(rf"\*{speaker_prefix}:\s*", "", line)
            cleaned_line = re.sub(r"•\d+_\d+•", "", cleaned_line)
            cleaned_line = cleaned_line.strip()
            tokens = re.findall(r"\b\w+\b", cleaned_line)

            for token in tokens:
                token_lower = token.lower()
                if token_lower in morpheme_dictionary:
                    morpheme_count = morpheme_dictionary[token_lower]
                    morpheme_counts_in_file.append(morpheme_count)

    if morpheme_counts_in_file:
        avg_morpheme_count = np.mean(morpheme_counts_in_file)
        std_morpheme_count = np.std(morpheme_counts_in_file)
        file_results.append((filename, avg_morpheme_count, std_morpheme_count))
        all_morpheme_counts.extend(morpheme_counts_in_file)
    else:
        file_results.append((filename, 0, 0))

# Вывод результатов по каждому файлу
for fname, avg_morph, std_morph in file_results:
    print(f"{fname}: среднее количество морфем = {avg_morph:.2f}, SD = {std_morph:.2f}")

# Общие значения по всем файлам
if all_morpheme_counts:
    overall_avg = np.mean(all_morpheme_counts)
    overall_sd = np.std(all_morpheme_counts)
    print(f"\nСреднее количество морфем на токен по всем файлам: {overall_avg:.2f}")
    print(f"Стандартное отклонение количества морфем по всем файлам: {overall_sd:.2f}")
else:
    print("Нет токенов, найденных в словаре морфем.")

Найдено 16 файлов в папке files.
TNK_S5.cha: среднее количество морфем = 1.44, SD = 0.71
FFK_S1.cha: среднее количество морфем = 1.48, SD = 0.65
SAP_S1.cha: среднее количество морфем = 1.46, SD = 0.80
ZVH_S1.cha: среднее количество морфем = 1.48, SD = 0.76
AVM_S1.cha: среднее количество морфем = 1.43, SD = 0.73
SAK_S2.cha: среднее количество морфем = 1.49, SD = 0.73
IMK_S3.cha: среднее количество морфем = 1.50, SD = 0.79
TYL_S1.cha: среднее количество морфем = 1.39, SD = 0.67
VVG_S1.cha: среднее количество морфем = 1.39, SD = 0.66
ENL_S1.cha: среднее количество морфем = 1.42, SD = 0.71
NVM_S3.cha: среднее количество морфем = 1.48, SD = 0.75
EKS_S4.cha: среднее количество морфем = 1.47, SD = 0.71
GAA_S1.cha: среднее количество морфем = 1.38, SD = 0.72
AVG_S2.cha: среднее количество морфем = 1.46, SD = 0.71
LNI_S5.cha: среднее количество морфем = 1.61, SD = 0.83
EAB_S9.cha: среднее количество морфем = 1.48, SD = 0.65

Среднее количество морфем на токен по всем файлам: 1.47
Стандартное от

In [14]:
import os
import glob
import re
import numpy as np
import pymorphy3

# Инициализация морфологического анализатора
morph_analyzer = pymorphy3.MorphAnalyzer()

# Путь к папке с .cha файлами
folder_path = "files"
file_list = glob.glob(os.path.join(folder_path, "*.cha"))

# Результаты
file_results = []
unique_counts = []

for file_path in file_list:
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    filename = os.path.basename(file_path)
    speaker_prefix = filename[:3]

    unique_tokens = set()

    for line in lines:
        if line.startswith(f"*{speaker_prefix}:"):
            cleaned_line = re.sub(rf"\*{speaker_prefix}:\s*", "", line)
            cleaned_line = re.sub(r"•\d+_\d+•", "", cleaned_line)
            cleaned_line = cleaned_line.strip()

            tokens = re.findall(r'\b\w+\b', cleaned_line)

            for token in tokens:
                if token.strip():
                    parsed = morph_analyzer.parse(token.lower())[0]
                    normal_form = parsed.normal_form
                    pos_tag = str(parsed.tag)
                    token_info = f"{normal_form} {pos_tag}"
                    unique_tokens.add(token_info)

    unique_count = len(unique_tokens)
    file_results.append((filename, unique_count))
    unique_counts.append(unique_count)

# Вывод результатов по каждому файлу
for fname, uniq in file_results:
    print(f"{fname}: количество уникальных токенов (lemma+POS) = {uniq}")

# Подсчёт среднего и SD по всем файлам
if unique_counts:
    avg_unique = np.mean(unique_counts)
    std_unique = np.std(unique_counts)
    print(f"\nСреднее количество уникальных токенов по всем файлам: {avg_unique:.2f}")
    print(f"Стандартное отклонение количества уникальных токенов: {std_unique:.2f}")
else:
    print("Нет данных для анализа.")

TNK_S5.cha: количество уникальных токенов (lemma+POS) = 501
FFK_S1.cha: количество уникальных токенов (lemma+POS) = 589
SAP_S1.cha: количество уникальных токенов (lemma+POS) = 538
ZVH_S1.cha: количество уникальных токенов (lemma+POS) = 1086
AVM_S1.cha: количество уникальных токенов (lemma+POS) = 644
SAK_S2.cha: количество уникальных токенов (lemma+POS) = 635
IMK_S3.cha: количество уникальных токенов (lemma+POS) = 665
TYL_S1.cha: количество уникальных токенов (lemma+POS) = 705
VVG_S1.cha: количество уникальных токенов (lemma+POS) = 517
ENL_S1.cha: количество уникальных токенов (lemma+POS) = 627
NVM_S3.cha: количество уникальных токенов (lemma+POS) = 544
EKS_S4.cha: количество уникальных токенов (lemma+POS) = 544
GAA_S1.cha: количество уникальных токенов (lemma+POS) = 653
AVG_S2.cha: количество уникальных токенов (lemma+POS) = 683
LNI_S5.cha: количество уникальных токенов (lemma+POS) = 552
EAB_S9.cha: количество уникальных токенов (lemma+POS) = 535

Среднее количество уникальных токенов 

In [15]:
import os
import glob
import re
import pymorphy3
import numpy as np

# Инициализация морфологического анализатора
morph_analyzer = pymorphy3.MorphAnalyzer()

# Путь к папке с .cha файлами
folder_path = "files"
file_list = glob.glob(os.path.join(folder_path, "*.cha"))

# Результаты
file_results = []
all_unique_sets = []

for file_path in file_list:
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    filename = os.path.basename(file_path)
    speaker_prefix = filename[:3]

    unique_tokens = set()

    for line in lines:
        if line.startswith(f"*{speaker_prefix}:"):
            cleaned_line = re.sub(rf"\*{speaker_prefix}:\s*", "", line)
            cleaned_line = re.sub(r"•\d+_\d+•", "", cleaned_line)
            cleaned_line = cleaned_line.strip()

            tokens = re.findall(r'\b\w+\b', cleaned_line)

            for token in tokens:
                if token.strip():
                    parsed = morph_analyzer.parse(token.lower())[0]
                    normal_form = parsed.normal_form
                    pos_tag = str(parsed.tag)
                    token_info = normal_form
                    unique_tokens.add(token_info)

    unique_count = len(unique_tokens)
    file_results.append((filename, unique_count))
    all_unique_sets.append(unique_tokens)

# Вывод результатов по файлам
for fname, uniq in file_results:
    print(f"{fname}: количество уникальных лемм (lemma+POS) = {uniq}")

# Подсчёт среднего количества уникальных токенов по всем файлам
if all_unique_sets:
    avg_unique = sum(len(s) for s in all_unique_sets) / len(all_unique_sets)
    std_unique = np.std(unique_counts)
    print(f"\nСреднее количество уникальных лемм по всем файлам: {avg_unique:.2f}")
    print(f"Стандартное отклонение количества уникальных токенов: {std_unique:.2f}")
else:
    print("Нет данных для анализа.")

TNK_S5.cha: количество уникальных лемм (lemma+POS) = 411
FFK_S1.cha: количество уникальных лемм (lemma+POS) = 466
SAP_S1.cha: количество уникальных лемм (lemma+POS) = 426
ZVH_S1.cha: количество уникальных лемм (lemma+POS) = 850
AVM_S1.cha: количество уникальных лемм (lemma+POS) = 503
SAK_S2.cha: количество уникальных лемм (lemma+POS) = 512
IMK_S3.cha: количество уникальных лемм (lemma+POS) = 523
TYL_S1.cha: количество уникальных лемм (lemma+POS) = 579
VVG_S1.cha: количество уникальных лемм (lemma+POS) = 430
ENL_S1.cha: количество уникальных лемм (lemma+POS) = 507
NVM_S3.cha: количество уникальных лемм (lemma+POS) = 454
EKS_S4.cha: количество уникальных лемм (lemma+POS) = 435
GAA_S1.cha: количество уникальных лемм (lemma+POS) = 509
AVG_S2.cha: количество уникальных лемм (lemma+POS) = 572
LNI_S5.cha: количество уникальных лемм (lemma+POS) = 433
EAB_S9.cha: количество уникальных лемм (lemma+POS) = 431

Среднее количество уникальных лемм по всем файлам: 502.56
Стандартное отклонение количе

In [8]:
!pip install lexicalrichness

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.8/97.8 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lexicalrichness: filename=lexicalrichness-0.5.1-py3-none-any.whl size=15418 sha256=6325700baed0741cf41991fd21568133e9a868c6e425b6506d1b2af55f169ac5
  Stored in directory: /root/.cache/pip/wheels/bb/68/f4/1a32ae0aae29ce426b4e3c106a4e8e9c19fd13bcaff45c6a8e
Successfully built lexicalrichness


In [16]:
import os
import glob
import re
from lexicalrichness import LexicalRichness
import numpy as np

# Путь к папке с .cha файлами
folder_path = "files"
file_list = glob.glob(os.path.join(folder_path, "*.cha"))

file_results = []
mtld_values = []

for file_path in file_list:
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    filename = os.path.basename(file_path)
    speaker_prefix = filename[:3]

    tokens = []

    for line in lines:
        if line.startswith(f"*{speaker_prefix}:"):
            cleaned_line = re.sub(rf"\*{speaker_prefix}:\s*", "", line)
            cleaned_line = re.sub(r"•\d+_\d+•", "", cleaned_line)
            cleaned_line = cleaned_line.strip()
            line_tokens = re.findall(r"\b\w+\b", cleaned_line)
            tokens.extend(line_tokens)

    if len(tokens) >= 50:
        text_for_mtld = " ".join(tokens)
        lex = LexicalRichness(text_for_mtld)
        mtld_score = lex.mtld(threshold=0.72)
        mtld_values.append(mtld_score)
        file_results.append((filename, mtld_score))
    else:
        file_results.append((filename, None))  # недостаточно данных

# Вывод результатов по каждому файлу
for fname, mtld in file_results:
    if mtld is not None:
        print(f"{fname}: MTLD = {mtld:.2f}")
    else:
        print(f"{fname}: слишком мало токенов для расчёта MTLD (<50)")

# Среднее значение MTLD и стандартное отклонение по всем подходящим файлам
if mtld_values:
    avg_mtld = np.mean(mtld_values)
    std_mtld = np.std(mtld_values)
    print(f"\nСреднее значение MTLD по всем файлам: {avg_mtld:.2f}")
    print(f"Стандартное отклонение MTLD по всем файлам: {std_mtld:.2f}")
else:
    print("Недостаточно данных для расчёта MTLD.")

TNK_S5.cha: MTLD = 48.21
FFK_S1.cha: MTLD = 80.89
SAP_S1.cha: MTLD = 47.76
ZVH_S1.cha: MTLD = 74.44
AVM_S1.cha: MTLD = 67.83
SAK_S2.cha: MTLD = 64.33
IMK_S3.cha: MTLD = 54.05
TYL_S1.cha: MTLD = 69.46
VVG_S1.cha: MTLD = 62.24
ENL_S1.cha: MTLD = 55.62
NVM_S3.cha: MTLD = 71.11
EKS_S4.cha: MTLD = 69.40
GAA_S1.cha: MTLD = 70.16
AVG_S2.cha: MTLD = 73.48
LNI_S5.cha: MTLD = 45.08
EAB_S9.cha: MTLD = 54.82

Среднее значение MTLD по всем файлам: 63.06
Стандартное отклонение MTLD по всем файлам: 10.49


In [17]:
import os
import re
import glob
from lexicalrichness import LexicalRichness
import pymorphy3
import numpy as np

# Морфоанализатор
morph = pymorphy3.MorphAnalyzer()

# Папка с .cha файлами
folder_path = "files"
file_list = glob.glob(os.path.join(folder_path, "*.cha"))

file_results = []
mtld_values = []

for file_path in file_list:
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    filename = os.path.basename(file_path)
    speaker_prefix = filename[:3]

    lemmas = []

    for line in lines:
        if line.startswith(f"*{speaker_prefix}:"):
            cleaned_line = re.sub(rf"\*{speaker_prefix}:\s*", "", line)
            cleaned_line = re.sub(r"•\d+_\d+•", "", cleaned_line)
            cleaned_line = cleaned_line.strip()
            tokens = re.findall(r"\b\w+\b", cleaned_line)

            for token in tokens:
                parsed = morph.parse(token)[0]
                lemmas.append(parsed.normal_form)

    if len(lemmas) >= 50:
        text_for_mtld = " ".join(lemmas)
        lex = LexicalRichness(text_for_mtld)
        mtld_score = lex.mtld(threshold=0.72)
        mtld_values.append(mtld_score)
        file_results.append((filename, mtld_score))
    else:
        file_results.append((filename, None))

# Вывод результатов
for fname, mtld in file_results:
    if mtld is not None:
        print(f"{fname}: MTLD по леммам = {mtld:.2f}")
    else:
        print(f"{fname}: недостаточно лемм для расчёта MTLD (<50)")

# Среднее значение и стандартное отклонение MTLD по всем подходящим файлам
if mtld_values:
    avg_mtld = np.mean(mtld_values)
    std_mtld = np.std(mtld_values)
    print(f"\nСреднее MTLD по леммам по всем файлам: {avg_mtld:.2f}")
    print(f"Стандартное отклонение MTLD по леммам по всем файлам: {std_mtld:.2f}")
else:
    print("Недостаточно данных для расчёта MTLD.")

TNK_S5.cha: MTLD по леммам = 30.75
FFK_S1.cha: MTLD по леммам = 47.05
SAP_S1.cha: MTLD по леммам = 31.19
ZVH_S1.cha: MTLD по леммам = 51.59
AVM_S1.cha: MTLD по леммам = 49.19
SAK_S2.cha: MTLD по леммам = 38.39
IMK_S3.cha: MTLD по леммам = 30.54
TYL_S1.cha: MTLD по леммам = 41.34
VVG_S1.cha: MTLD по леммам = 35.29
ENL_S1.cha: MTLD по леммам = 34.73
NVM_S3.cha: MTLD по леммам = 52.37
EKS_S4.cha: MTLD по леммам = 42.06
GAA_S1.cha: MTLD по леммам = 40.98
AVG_S2.cha: MTLD по леммам = 45.16
LNI_S5.cha: MTLD по леммам = 35.23
EAB_S9.cha: MTLD по леммам = 33.48

Среднее MTLD по леммам по всем файлам: 39.96
Стандартное отклонение MTLD по леммам по всем файлам: 7.18
